## Setup

In [ ]:
import os
import sys

BASE_PATH = "/home/MD00560695/workdir/gradcam"
sys.path.append(BASE_PATH)

In [2]:
DATA_PATH = os.path.join(BASE_PATH, "data/raw/tiny-imagenet-200/")

In [3]:
sample_image = os.path.join(DATA_PATH, "val/images/val_10.JPEG") #"val/cat_dog.png")
SAMPLE_DIR = os.path.join(DATA_PATH, "val/images") # "imgs/samples")
MODEL_DIR= os.path.join(BASE_PATH, "models")
logo_img = os.path.join(BASE_PATH, "imgs", "alaiom_07.JPG")
LOG = True

In [6]:
0 % 1

0

## Imports

In [4]:
import tensorflow as tf
from tensorflow.python.framework import ops
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import load_img, img_to_array
# Models architecture
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.applications.resnet_v2 import preprocess_input, decode_predictions
from tensorflow.keras.layers import Dense, Flatten, Activation
from tensorflow.keras.applications.resnet50 import ResNet50

In [5]:
import cv2
import re
import numpy as np
from PIL import Image
from io import BytesIO

In [6]:
from IPython.display import display, Javascript, HTML, clear_output, IFrame
from ipywidgets import interact, interactive, fixed, interact_manual, AppLayout, GridspecLayout
import ipywidgets as widgets
import matplotlib.pyplot as plt

## Helper Functions

In [7]:
TARGET_SIZE = (224,224)

In [8]:
def load_image(path, preprocess=True):
    """Load and preprocess image."""
    x = image.load_img(path, target_size=(H, W))
    if preprocess:
        x = image.img_to_array(x)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
    return x

In [9]:
def deprocess_image(x):
    """Same normalization as in:
    https://github.com/fchollet/keras/blob/master/examples/conv_filter_visualization.py
    """
    # normalize tensor: center on 0., ensure std is 0.25
    x = x.copy()
    x -= x.mean()
    x /= (x.std() + K.epsilon())
    x *= 0.25

    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to RGB array
    x *= 255
    if K.image_data_format() == 'channels_first':
        x = x.transpose((1, 2, 0))
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [10]:
def deprocess(x):
    if np.ndim(x) > 3:
        x = np.squeeze(x)
        
    with tf.GradientTape() as tape:
        x = -tf.math.reduce_mean(x)
        x /= (tf.math.reduce_std(x) + 1e-5) 
        x *= 0.1

        # clip to [0, 1]
        x += 0.5
        x = np.clip(x, 0, 1)

        # convert to RGB array
        x *= 25
        if K.image_data_format() == 'th':
            x = x.transpose((1, 2, 0))
            
        x = np.clip(x, 0, 255).astype('uint8')
        
    return x

In [11]:
def normalize(x):
    """Utility function to normalize a tensor by its L2 norm"""
    return (x + 1e-10) / (K.sqrt(K.mean(K.square(x))) + 1e-10)

In [12]:
def array2bytes(im_arr, fmt='png'):
    img = Image.fromarray(im_arr, mode='RGB')
    f = BytesIO()
    img.save(f, fmt)

    return f.getvalue()

In [13]:
def preprocess(filename):
    im = img_to_array(load_img(os.path.join(SAMPLE_DIR,filename),target_size = TARGET_SIZE))
    x = np.expand_dims(im, axis=0)
    x = preprocess_input(x)

    return x

## Model Definition

In [14]:
class VanilaResNet50:
    def __init__(self, n_classes=1000):
        self.n_classes = n_classes

    def __call__(self):
        resnet = ResNet50V2(include_top=False, pooling="avg", weights='imagenet')
        for layer in resnet.layers:
            layer.trainable = False

        logits = Dense(self.n_classes)(resnet.layers[-1].output)
        output = Activation('softmax')(logits)
        model = Model(resnet.input, output)

        return model

In [15]:
class ResNet50PlusFC:
    def __init__(self, n_classes=1000):
        resnet = ResNet50V2(include_top=False, pooling="avg", weights='imagenet')
        for layer in resnet.layers:
            layer.trainable = False

        fc1 = Dense(100)(resnet.layers[-1].output)
        fc2 = Dense(100)(fc1)
        logits = Dense(n_classes)(fc2)
        output = Activation('softmax')(logits)
        model = Model(resnet.input, output)

        return model

In [16]:
def load_VanilaResNet50(n_classes=1000):
    resnet = ResNet50V2(include_top=False, pooling="avg", weights='imagenet')
    for layer in resnet.layers:
        layer.trainable = False

    logits = Dense(n_classes)(resnet.layers[-1].output)
    output = Activation('softmax')(logits)
    model = Model(resnet.input, output)
    #model.load_weights("{}/resnet50best.hdf5".format(MODEL_DIR))

    return model

In [17]:
def load_ResNet50PlusFC(n_classes=1000):
    resnet = ResNet50V2(include_top=False, pooling="avg", weights='imagenet')
    for layer in resnet.layers:
        layer.trainable = False

    fc1 = Dense(100)(resnet.layers[-1].output)
    fc2 = Dense(100)(fc1)
    logits = Dense(n_classes)(fc2)
    output = Activation('softmax')(logits)
    model = Model(resnet.input, output)
    #model.load_weights("{}/resnet50fcbest.hdf5".format(MODEL_DIR))

    return model

In [18]:
def load_ResNet50():
    return ResNet50(include_top=True, weights='imagenet')

In [19]:
def predict(model, processed_im):
    top_n = 5
    preds = model.predict(processed_im)
    
    top_pred_n = decode_predictions(preds, top=top_n)[0]
    classes = np.argsort(preds[0])[-top_n:][::-1]
        
    P = imagenet_utils.decode_predictions(preds)
    idx = preds.argmax()
    return idx, preds.max(), classes, top_pred_n

## GRAD CAM

In [20]:
class GradCAM:
    # Adapted with some modification from 
    # https://www.pyimagesearch.com/2020/03/09/grad-cam-visualize-class-activation-maps-with-keras-tensorflow-and-deep-learning/
    def __init__(self, model, layerName=None):
        """
        model: pre-softmax layer (logit layer)
        """
        self.model = model
        self.layerName = layerName
        self.cam3 = None

        if self.layerName == None:
            self.layerName = self.find_target_layer()

    def find_target_layer(self):
        for layer in reversed(self.model.layers):
            if len(layer.output_shape) == 4:
                return layer.name
        raise ValueError("Could not find 4D layer. Cannot apply GradCAM")

    def compute_heatmap(self, image, classIdx, upsample_size, eps=1e-5):
        gradModel = Model(
            inputs=[self.model.inputs],
            outputs=[self.model.get_layer(self.layerName).output, self.model.output]
        )
        # record operations for automatic differentiation

        with tf.GradientTape() as tape:
            inputs = tf.cast(image, tf.float32)
            (convOuts, preds) = gradModel(inputs)  # preds after softmax
            loss = preds[:, classIdx]

        # compute gradients with automatic differentiation
        grads = tape.gradient(loss, convOuts)
        # discard batch
        convOuts = convOuts[0]
        grads = grads[0]
        norm_grads = tf.divide(grads, tf.reduce_mean(tf.square(grads)) + tf.constant(eps))

        # compute weights
        weights = tf.reduce_mean(norm_grads, axis=(0, 1))
        cam = tf.reduce_sum(tf.multiply(weights, convOuts), axis=-1)

        # Apply reLU
        cam = np.maximum(cam, 0)
        cam = cam / np.max(cam)
        cam = cv2.resize(cam, upsample_size,cv2.INTER_LINEAR)

        # convert to 3D
        cam3 = np.expand_dims(cam, axis=2)
        cam3 = np.tile(cam3, [1, 1, 3])
        
        self.cam3 = cam3
        return cam3
    
    
    def overlay_gradCAM(self, img):
        superimposed_cam3 = np.uint8(255 * self.cam3)
        superimposed_cam3 = cv2.applyColorMap(superimposed_cam3, cv2.COLORMAP_JET)

        superimposed_img = 0.3 * superimposed_cam3 + 0.5 * img

        return (superimposed_img * 255.0 / superimposed_img.max()).astype("uint8")

## Guided Backpropogation

In [21]:
@tf.custom_gradient
def guidedRelu(x):
    def grad(dy):
        return tf.cast(dy>0,"float32") * tf.cast(x>0, "float32") * dy
    return tf.nn.relu(x), grad

In [22]:
#Reference: https://github.com/eclique/keras-gradcam with adaption to tensorflow 2.0  
class GuidedBackprop:
    def __init__(self,model, layerName=None):
        self.model = model
        self.layerName = layerName
        if self.layerName == None:
            self.layerName = self.find_target_layer()
        self.gbModel = self.build_guided_model()
        
    def find_target_layer(self):
        for layer in reversed(self.model.layers):
            if len(layer.output_shape) == 4:
                return layer.name
        raise ValueError("Could not find 4D layer. Cannot apply Guided Backpropagation")

    def build_guided_model(self):
        gbModel = Model(
            inputs = [self.model.inputs],
            outputs = [self.model.get_layer(self.layerName).output]
        )
        layer_dict = [layer for layer in gbModel.layers[1:] if hasattr(layer,"activation")]
        for layer in layer_dict:
            if layer.activation == tf.keras.activations.relu:
                layer.activation = guidedRelu
        
        return gbModel
    
    def guided_backprop(self, images, upsample_size):
        """Guided Backpropagation method for visualizing input saliency."""
        with tf.GradientTape() as tape:
            inputs = tf.cast(images, tf.float32)
            tape.watch(inputs)
            outputs = self.gbModel(inputs)

        grads = tape.gradient(outputs, inputs)[0]

        saliency = cv2.resize(np.asarray(grads), upsample_size)

        return saliency

## Build IPywidget UI

In [23]:
# Header
header = widgets.HTML('<font color="#1f77b4" face="sans-serif"><center><h1>DEMO GradCAM and Guided GradCAM</h1></center></font>',
                      layout=widgets.Layout(height='auto'))

In [24]:
# Logo
logo = widgets.Image(
    value=open(logo_img, "rb").read(),
    format='png',
    width='auto',
    height='auto',
    align="center-align"
)

In [25]:
def setclasseswidget(class_ls):
    CLASSES = widgets.Dropdown(options=class_ls, description="Class", layout={'width':'auto'}, disabled=False)
    grid[1,17:24] = CLASSES

In [26]:
# Dropdown for Image
def on_change_im(change):
    if change['type'] == "change" and change["name"] == "value":
        img = cv2.imread(os.path.join(SAMPLE_DIR,change["new"]))
        im_arr = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        logo = widgets.Image(
            value=array2bytes(im_arr),
            format='png',
            width='auto',
            height='auto',
            align="center-align"
        )
        grid[1,17:24] = widgets.HTML("")
        grid[5:13, 1:9] = widgets.HTML("")
        grid[5:13, 11:19] = logo
        grid[5:13, 21:29] = widgets.HTML("")

In [27]:
# Dropdown for model
def on_change_model(change):
    if change['type'] == "change" and change["name"] == "value":
        chosen_model =  widgets.HTML("<center><p>Model %s loaded.<center>" % change["new"])
        grid[2,:8] = chosen_model

In [28]:
# button
def create_expanded_button(description, button_style):
    return widgets.Button(description=description, button_style=button_style,
                          layout=widgets.Layout(height='auto', width='auto'))

In [29]:
# set the drop down values
im_ls = ["--Select"] + os.listdir(SAMPLE_DIR)
im_ls.sort()
model_ls = ["--Select","VanilaResNet50", "ResNet50PlusFC"]
class_ls = ["--Select"]

In [30]:
# set the component values
CLASSES = widgets.Dropdown(options=class_ls, description="Class", layout={'width':'auto'}, disabled=False)
models = widgets.Dropdown(options=model_ls, description="Model",layout={'width':'auto'}, disabled=False)
imgs = widgets.Dropdown(options=im_ls, description="Image", layout={'width':'auto'}, disabled=False)
imgs.observe(on_change_im)
models.observe(on_change_model)

In [31]:
# Set button values
pred_but = create_expanded_button("Show","info")

In [32]:
# Set Grid Layouts
grid = GridspecLayout(20, 30, height='700px')
grid[0,:] = header
grid[1,:8] = models
grid[1,8:17] = imgs
grid[1,25:] = pred_but
grid[5:13,11:19] = logo

## Define Functionalities 

In [33]:
def showCAMs(img, x, GradCAM, GuidedBP, chosen_class, upsample_size):
    # Grad CAM
    log_info("showCAMs","Computing gradcam")
    cam3 = GradCAM.compute_heatmap(image=x, classIdx=chosen_class, upsample_size=upsample_size)
    gradcam = GradCAM.overlay_gradCAM(img)
    gradcam = cv2.cvtColor(gradcam, cv2.COLOR_BGR2RGB)
    
    # Guided backprop
    log_info("showCAMs","Computing backprop")
    gb = GuidedBP.guided_backprop(x, upsample_size)
    gb_im = deprocess_image(gb)
    gb_im = cv2.cvtColor(gb_im, cv2.COLOR_BGR2RGB)
    
    # Guided GradCAM
    log_info("showCAMs","Computing guidied gradcam")
    guided_gradcam = deprocess_image(gb*cam3)
    guided_gradcam = cv2.cvtColor(guided_gradcam, cv2.COLOR_BGR2RGB)
    
    # Display
    log_info("showCAMs","Display images")
    gc = widgets.Image(
            value=array2bytes(gradcam),
            format='png',
            width='auto',
            height='auto',
            align="center-align"
        )
    gbim = widgets.Image(
            value=array2bytes(gb_im),
            format='png',
            width='auto',
            height='auto',
            align="center-align"
        )
    ggc = widgets.Image(
            value=array2bytes(guided_gradcam),
            format='png',
            width='auto',
            height='auto',
            align="center-align"
        )
    
    log_info("showCAMs","Set new Grid")
    grid[4, 1:9] = widgets.HTML('<center><b>GradCAM</b></center>')
    grid[4, 11:19] = widgets.HTML('<center><b>Guided Bacpropagation</b></center>')
    grid[4, 21:29] = widgets.HTML('<center><b>Guided GradCAM</b></center>')
    grid[5:13, 1:9] = gc
    grid[5:103, 11:19] = gbim
    grid[5:13, 21:29] = ggc

In [34]:
def check_button(sender):
    log_info("check_button","enter function")
    if models.value == "VanilaResNet50":
        model = load_VanilaResNet50()
        gradCAM = GradCAM(model=model, layerName="conv5_block3_out")
        guidedBP = GuidedBackprop(model=model,layerName="conv5_block3_out")
        log_info("check_button","set vanila function")
    elif models.value == "ResNet50PlusFC":
        model = load_ResNet50PlusFC()
        gradCAM = GradCAM(model=model, layerName="conv5_block3_out")
        guidedBP = GuidedBackprop(model=model, layerName="conv5_block3_out")
    
    # read image
    img = cv2.imread(os.path.join(SAMPLE_DIR,imgs.value))
    log_info("check_button","read sample image")
    upsample_size = (img.shape[1],img.shape[0])
    x = preprocess(imgs.value)
    log_info("check_button","Completed image processing")
    pred, prob, predclasses, top_5_pred = predict(model,x)
    class_ls = ["--Select"]
    for c, p in zip(predclasses, top_5_pred):
        class_ls.append("{} ({}) {:.3f}".format(p[1], c, p[2]))
        
    CLASSES = widgets.Dropdown(options=class_ls, description="Class", layout={'width':'auto'}, disabled=False)
    grid[1,17:24] = CLASSES
    log_info("check_button","prediction completed")
    #log_info("check_button","Checking classvalue {}/{}".format(classes.value, INV_MAP[classes.value]))
    if CLASSES.value == "--Select":
        classIdx = pred
    else:
        classIdx = getclassidx(CLASSES.value)
    
    grid[2,9:18] = widgets.HTML("<center><span>Predicted Class: <b>{}<b> probability: <b>{:.3f}<b><span><center>".format(pred, prob)) 
    log_info("check_button","set grid")
    showCAMs(img, x, gradCAM, guidedBP, classIdx, upsample_size)
    log_info("check_button","completed")

In [35]:
def getclassidx(value):
    result = re.search(r"\[([0-9]+)\]", value)
    return result

In [36]:
def log_info(function="main", message=""):
    if LOG:
        print("{}\t: {}".format(function, message))

In [37]:
pred_but.on_click(check_button)

In [38]:
# Display the grid
display(grid)

GridspecLayout(children=(HTML(value='<font color="#1f77b4" face="sans-serif"><center><h1>DEMO GradCAM and Guid…